<a href="https://colab.research.google.com/github/Amiya-Kalita/Machine-Learning/blob/main/Loan_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Testing is Gpu Availabel For Training

In [1]:
import torch
import torch.nn as nn

if torch.cuda.is_available():
    device  = torch.device('cuda')
    print(f"Gpu: {torch.cuda.get_device_name(0)} is available . using device : {device}")
else:
    device = torch.device('cpu')
    print(f"No Gpu are available . using device : {device}")

Gpu: Tesla T4 is available . using device : cuda


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#### Loding Dataset for model evaluation

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/DataSets/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DataSets/test.csv')

In [4]:
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


#### Performing eda on the training dataset

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.5,3812.5,5795.00,81000.0
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.0,1188.5,2297.25,41667.0
LoanAmount,592.0,146.412162,85.587325,9.0,100.0,128.0,168.00,700.0
Loan_Amount_Term,600.0,342.000000,65.120410,12.0,360.0,360.0,360.00,480.0
Credit_History,564.0,0.842199,0.364878,0.0,1.0,1.0,1.00,1.0


In [7]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score

In [8]:
train_df = train_df.drop(columns= ['Loan_ID'])
test_df = test_df.drop(columns = ["Loan_ID"])

train_df ['Loan_Status'] = train_df['Loan_Status'].map({'Y': 1 , 'N': 0})

In [10]:
train_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [11]:
x = train_df.drop(columns= ['Loan_Status'])
y = train_df['Loan_Status'].values

cat_cols = x.select_dtypes(include=['object']).columns.tolist()
num_cols = x.select_dtypes(exclude=['object']).columns.tolist()

In [13]:
preprocessor  = ColumnTransformer([
    ('num' , Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols),

    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), cat_cols)
])

In [15]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

In [16]:
class LoanDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.toarray() if hasattr(X, 'toarray') else X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = LoanDataset(X_train, y_train)
val_ds = LoanDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128)

In [17]:
class LoanModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(1)

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LoanModel(X_train.shape[1]).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [19]:
for epoch in range(50):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    preds, actuals = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            out = torch.sigmoid(model(xb)).cpu().numpy()
            preds.extend(out)
            actuals.extend(yb.numpy())

    acc = accuracy_score(actuals, np.array(preds) > 0.5)
    auc = roc_auc_score(actuals, preds)
    print(f"Epoch {epoch+1:02d} | Loss: {train_loss/len(train_loader):.4f} | Acc: {acc:.4f} | AUC: {auc:.4f}")

Epoch 01 | Loss: 0.7175 | Acc: 0.7642 | AUC: 0.7285
Epoch 02 | Loss: 0.6591 | Acc: 0.6911 | AUC: 0.7443
Epoch 03 | Loss: 0.6192 | Acc: 0.6992 | AUC: 0.7551
Epoch 04 | Loss: 0.5860 | Acc: 0.7805 | AUC: 0.7737
Epoch 05 | Loss: 0.5541 | Acc: 0.8455 | AUC: 0.8096
Epoch 06 | Loss: 0.5244 | Acc: 0.8537 | AUC: 0.8300
Epoch 07 | Loss: 0.4971 | Acc: 0.8618 | AUC: 0.8412
Epoch 08 | Loss: 0.4940 | Acc: 0.8618 | AUC: 0.8424
Epoch 09 | Loss: 0.4899 | Acc: 0.8618 | AUC: 0.8455
Epoch 10 | Loss: 0.4925 | Acc: 0.8618 | AUC: 0.8455
Epoch 11 | Loss: 0.4816 | Acc: 0.8618 | AUC: 0.8430
Epoch 12 | Loss: 0.4654 | Acc: 0.8618 | AUC: 0.8430
Epoch 13 | Loss: 0.4692 | Acc: 0.8618 | AUC: 0.8409
Epoch 14 | Loss: 0.4734 | Acc: 0.8618 | AUC: 0.8409
Epoch 15 | Loss: 0.4827 | Acc: 0.8618 | AUC: 0.8455
Epoch 16 | Loss: 0.4694 | Acc: 0.8618 | AUC: 0.8474
Epoch 17 | Loss: 0.4631 | Acc: 0.8618 | AUC: 0.8452
Epoch 18 | Loss: 0.4719 | Acc: 0.8618 | AUC: 0.8437
Epoch 19 | Loss: 0.4674 | Acc: 0.8618 | AUC: 0.8437
Epoch 20 | L

In [21]:
model.eval()
with torch.no_grad():
    preds = torch.sigmoid(model(torch.tensor(X_val, dtype=torch.float32).to(device))).cpu().numpy()
print("Validation Accuracy:", accuracy_score(y_val, preds>0.5))
print("Validation AUC:", roc_auc_score(y_val, preds))

Validation Accuracy: 0.8617886178861789
Validation AUC: 0.834984520123839


In [23]:
X_test = preprocessor.transform(test_df)
X_test_torch = torch.tensor(X_test, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    test_preds = torch.sigmoid(model(X_test_torch)).cpu().numpy()

submission = pd.DataFrame({
    'Loan_ID': pd.read_csv("/content/drive/MyDrive/DataSets/test.csv")['Loan_ID'],
    'Loan_Status': np.where(test_preds>0.5, 'Y', 'N')
})
submission.to_csv("submission.csv", index=False)
print(" Submission file saved as submission.csv")

✅ Submission file saved as submission.csv
